# UDP port scan utilities

This notebook offers lightweight ways to probe UDP ports:
- probe_udp: send a UDP packet and wait for an application-layer response (works if the service replies)
- scan_ports: run probe_udp over a list/range of ports
- run_nmap_udp: wrapper to call `nmap -sU` if nmap is installed (recommended for thorough scans)

Notes: UDP scanning is inherently unreliable: lack of response means "open|filtered"; detecting closed ports often requires listening for ICMP Port Unreachable messages which needs admin/root privileges.


In [ ]:
import socket, subprocess, sys, shutil
from typing import Iterable, Union

def probe_udp(host: str, port: int, timeout: float = 1.0, payload: bytes = b'') -> str:
    """Send a UDP packet and wait for a response.
    Returns:
      'open (response)' if a UDP reply was received,
      'open|filtered (no response)' if no reply (could be open or filtered),
      'error:...' on socket errors.
    Note: absence of response does NOT prove closed.
    """
    s = None
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.settimeout(timeout)
        s.sendto(payload, (host, port))
        # some UDP services reply (e.g., DNS on 53)
        data, addr = s.recvfrom(4096)
        return 'open (response) - {} bytes from {}'.format(len(data), addr)
    except socket.timeout:
        return 'open|filtered (no response)'
    except Exception as e:
        return f'error:{e}'
    finally:
        if s:
            try:
                s.close()
            except:
                pass

def scan_ports(host: str, ports: Union[Iterable[int], range], timeout: float = 1.0) -> dict:
    "Scan several UDP ports using probe_udp. Returns a dict port->result."
    results = {}
    for p in ports:
        results[p] = probe_udp(host, p, timeout=timeout)
    return results

def run_nmap_udp(host: str, ports: Union[str, None] = None) -> str:
    "Call nmap -sU if nmap is available. `ports` can be a string like '1-1024' or None for default."
    nmap = shutil.which('nmap')
    if not nmap:
        return 'nmap not found on PATH'
    cmd = [nmap, '-sU', host]
    if ports:
        cmd = [nmap, '-sU', '-p', str(ports), host]
    try:
        p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, timeout=300)
        return p.stdout + '
' + p.stderr
    except Exception as e:
        return f'error running nmap: {e}'

# Admin-only ICMP-based listener is possible but platform-dependent; see notes above.


In [ ]:
# Example usage: scan localhost on a few common UDP ports
if __name__ == '__main__':
    target = '127.0.0.1'
    ports = [53, 123, 161, 12345]  # DNS, NTP, SNMP, and a high port
    print('Scanning', target, 'ports', ports)
    res = scan_ports(target, ports, timeout=1.0)
    for p, r in res.items():
        print(p, r)
    # Try nmap if installed:
    print('
Trying nmap (if installed)...')
    print(run_nmap_udp(target, '1-200'))
